<h4>Imports</h4>

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import Imputer
from sklearn.metrics import make_scorer


<h4>Data Import and Cleaning</h4>

In [4]:
#reading and formatting part 1
df = pd.read_csv('data/Train.csv')
df.columns = np.array([(str(x)).lower() for x in df.columns])
df = df[['modelid', 'auctioneerid', 'yearmade', 'machinehourscurrentmeter', \
        'saledate', 'saleprice', 'state', 'productgroup']]
df['sale_month'] = pd.to_datetime(df['saledate'], infer_datetime_format = True).dt.month
df['sale_year'] = pd.to_datetime(df['saledate'], infer_datetime_format = True).dt.year
df.drop('saledate', axis = 1, inplace = True)
df.saleprice = df.saleprice.astype(float)

/Users/Trent/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#formatting part 2
df_dummies = pd.get_dummies(df[['state', 'productgroup']])
df = pd.concat([df_dummies, df], axis=1)
df.drop(['state', 'productgroup', 'machinehourscurrentmeter'], axis = 1, inplace = True)

<h4>Get Train Test Split Data, Train Model</h4>

In [6]:
y = df.pop('saleprice')
X = df.values

In [7]:
# #Impute machinehourscurrentmeter using the mean
# pute = Imputer(copy = False, strategy = 'mean')
# machine = np.array(pute.fit_transform(X[:, -3].reshape(-1,1)))
# machine = machine.reshape(1, len(machine))
# X[:,-3] = machine

#Impute auctioneerid, 
pute2 = Imputer(copy = False, strategy = 'most_frequent')
auctioneer = np.array(pute2.fit_transform(X[:, -5].reshape(-1,1)))
auctioneer = auctioneer.reshape(1, len(auctioneer))
X[:,-4] = auctioneer

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
print df.info()
print '\n'
print X[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 64 columns):
state_Alabama           401125 non-null float64
state_Alaska            401125 non-null float64
state_Arizona           401125 non-null float64
state_Arkansas          401125 non-null float64
state_California        401125 non-null float64
state_Colorado          401125 non-null float64
state_Connecticut       401125 non-null float64
state_Delaware          401125 non-null float64
state_Florida           401125 non-null float64
state_Georgia           401125 non-null float64
state_Hawaii            401125 non-null float64
state_Idaho             401125 non-null float64
state_Illinois          401125 non-null float64
state_Indiana           401125 non-null float64
state_Iowa              401125 non-null float64
state_Kansas            401125 non-null float64
state_Kentucky          401125 non-null float64
state_Louisiana         401125 non-null float64
state_Maine             4

In [9]:
gb = GradientBoostingRegressor()
gb.fit(X_train, y_train)
score = gb.score(X_test, y_test)

In [10]:
log_score = cross_val_score(gb, X_test, y_test, scoring='mean_squared_error', cv=5, n_jobs=-1)

/Users/Trent/anaconda/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/Trent/anaconda/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/Trent/anaconda/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/Trent/anaconda/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/T

In [11]:
print "R_squared:", score
print "Mean Error:", np.sqrt(-1*log_score)

R_squared: 0.628563974236
Mean Error: [ 14041.42453832  14307.13839782  14158.0166562   13975.87656449
  14457.04262696]
